In [1]:
# Importing the libraries 

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
# Importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt #visualisation
%matplotlib inline
sns.set(color_codes=True) 
# Load the libraries
import numpy as np 
import pandas as pd 
import seaborn as sns
import missingno as msno
import re
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from warnings import filterwarnings
filterwarnings('ignore')


from scipy.stats.mstats import winsorize
import scipy.stats as stats
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,explained_variance_score
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import (GridSearchCV, cross_val_score, cross_val_predict, StratifiedKFold, learning_curve)

from statsmodels.tools.eval_measures import mse, rmse
from sklearn import preprocessing

In [2]:
df_compare = pd.read_excel('milk_compare.xlsx')
df_compare.head(12)

,Survey Date,Ireland,Germany,Italy
0,January,46.03,41.66,41.5
1,February,47.59,43.12,42.5
2,March,48.95,44.86,43.5
3,April,51.28,47.20,48.0
4,May,53.61,49.59,48.0
5,June,55.45,52.08,48.5
6,July,57.78,55.04,55.0
7,August,59.73,56.77,55.0
8,September,64.29,58.18,57.0
9,October,68.47,59.33,57.0


In [3]:
df_compare = df_compare.dropna()
df_compare.isnull().sum()

Survey Date    0
Ireland        0
Germany        0
Italy          0
dtype: int64

In [4]:
df_compare.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Survey Date  12 non-null     object 
 1   Ireland      12 non-null     float64
 2   Germany      12 non-null     float64
 3   Italy        12 non-null     float64
dtypes: float64(3), object(1)
memory usage: 480.0+ bytes


## One-way Analysis of Variance (ANOVA)

Мы хотим увидеть, превосходит ли цена одной страны -  другой, поэтому наша нулевая гипотеза состоит в том, что средний цена в каждой группе эквивалентен средней цене в других группах. Для простоты мы рассмотрим 3 группы (страны «Ireland», «Germany», «Italy») по 12 месяцев в каждой.

In [5]:
a = df_compare['Ireland'].values
b = df_compare['Germany'].values
c = df_compare['Italy'].values

In [6]:
import scipy.stats as stats 
stats.f_oneway(a,b,c) 

F_onewayResult(statistic=2.563523709340257, pvalue=0.0922853211480521)

## Student's t-test for independent groups

The Student’s t-Test is a statistical hypothesis test to compare whether two samples are expected to have been drawn from the same population. It is named for the pseudonym “Student” used by William Gosset, who developed the test.

The test works by checking the means from two samples to see if they are significantly different from each other. It does this by calculating the standard error in the difference between means, which can be interpreted to see how likely the difference is, if the two samples have the same mean (the null hypothesis).

The t-statistic calculated by the test can be interpreted by comparing it to critical values from the t-distribution. The critical value can be calculated using the degrees of freedom and a significance level with the percent point function (PPF).

We can interpret the statistic value in a two-tailed test, meaning that if we reject the null hypothesis, it could be because the first mean is smaller or greater than the second mean.

We can implement a function as shown below:



In [7]:
import scipy.stats as st

In [8]:
def independent_ttest(a, b, alpha=0.05):
    # calculate means
    mean1, mean2 = np.mean(a), np.mean(b)
    # calculate standard errors
    se1, se2 = st.sem(a), st.sem(b)
    # standard error on the difference between the samples
    sed = np.sqrt(se1**2.0 + se2**2.0)
    # calculate the t statistic
    t_stat = (mean1 - mean2) / sed
    # degrees of freedom
    df = len(a) + len(b) - 2
    # calculate the critical value
    cv = st.t.ppf(1.0 - alpha, df)
    # calculate the p-value
    p = (1.0 - st.t.cdf(abs(t_stat), df)) * 2.0
    # return everything
    return t_stat, df, cv, p
    

In [9]:
# calculate the t test
alpha = 0.05
t_stat, df, cv, p = independent_ttest(a, b, alpha)
print('t=%.3f, df=%d, cv=%.3f, p=%.3f' % (t_stat, df, cv, p))
print()
# interpret via critical value
if abs(t_stat) <= cv:
    print('Fail to reject null hypothesis that the means are equal.')
else:
    print('Reject the null hypothesis that the means are equal.')
# interpret via p-value
if p > alpha:
    print('Fail to reject null hypothesis that the means are equal.')
else:
    print('Reject the null hypothesis that the means are equal.')

t=1.636, df=22, cv=1.717, p=0.116

Fail to reject null hypothesis that the means are equal.
Fail to reject null hypothesis that the means are equal.


# Non-Parametric Hypothesis t-Test
Wilcoxon Signed-Rank, Mann-Whitney U, Kruskal-Wallis H, Friedman Test

### Mann-Whitney U Test
Tests whether the distributions of two independent samples are equal or not. The same as the One-sample t-test.
Assumptions:

*Observations in each sample are independent & identically distributed(iid)

*Observations in each sample can be ranked.

In [10]:
from scipy.stats import mannwhitneyu

stat, p = mannwhitneyu(a, b)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
 print('Probably the same distribution')
else:
 print('Probably different distributions')

stat=95.000, p=0.194
Probably the same distribution


### Wilcoxon Signed-Rank Test
tests whether the distributions of two paired samples are equal or not. Sample like Paired t-Test

Assumptions:

Observations in each sample are independent & identically distributed (iid).
Observations in each sample can be ranked.
Observations across each sample are paired.

In [11]:
from scipy.stats import wilcoxon

stat, p = wilcoxon(a, c)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
 print('Probably the same distribution')
else:
 print('Probably different distributions')

stat=0.000, p=0.000
Probably different distributions


### Kruskal-Wallis H Test or One-way Anova
tests whether the distributions of two or more independent samples are equal or not.

What is the difference between Mann-Whitney and Kruskal-Wallis?

The major difference between the Mann-Whitney U and the Kruskal-Wallis H is simply that the latter can accommodate more than two groups.

The ANOVA (and t-test) is explicitly a test of equality of means of values. The Kruskal-Wallis (and Mann-Whitney) can be seen technically as a comparison of the mean ranks.

Assumptions:

Observations in each sample are independent and identically distributed (iid).
Observations in each sample can be ranked.



In [12]:
from scipy.stats import kruskal

stat, p = kruskal(a, b)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
 print('Probably the same distribution')
else:
 print('Probably different distributions')

stat=1.764, p=0.184
Probably the same distribution


### Friedman Test
tests whether the distributions of two or more paired samples are equal or not alternative to the Repeated Measures ANOVA.

It is used to determine whether or not there is a statistically significant difference between the means of three or more groups in which the same subjects show up in each group.

Assumptions:
Observations in each sample are independent and identically distributed (iid).
Observations in each sample can be ranked.
Observations across each sample are paired.

In [13]:
from scipy.stats import friedmanchisquare

stat, p = friedmanchisquare(a, b, c)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
 print('Probably the same distribution')
else:
 print('Probably different distributions')


stat=22.167, p=0.000
Probably different distributions
